In [1]:
# ------------------------------------------------------------------------------ #
# @Author:        F. Paul Spitzner
# @Email:         paul.spitzner@ds.mpg.de
# @Created:       2023-08-04 11:59:06
# @Last Modified: 2023-08-04 11:59:15
# ------------------------------------------------------------------------------ #
# ------------------------------------------------------------------------------ #

%reload_ext autoreload
%autoreload 2
%reload_ext ipy_dict_hierarchy
%reload_ext watermark

import logging
logging.basicConfig(
    format="%(asctime)s | %(levelname)-8s | %(name)-s > %(funcName)-s > %(message)s",
    level=logging.WARNING,
)
log = logging.getLogger("notebook")
log.setLevel("DEBUG")

import re
import glob
import h5py
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
import dask

from tqdm import tqdm


# also needs to be added for each dask-worker
extra_path = os.path.abspath('../')
sys.path.append(extra_path)
log.info(f"project directory: {extra_path}")

from ana import utility as utl
utl.log.setLevel("DEBUG")


2023-08-11 17:40:38,141 | INFO     | notebook > <module> > project directory: /Users/paul/para/2_Projects/information_timescales/repo/_latest/experiment_analysis


In [29]:
utl.log.setLevel("DEBUG")
# specify the path as closely as possible, we search recursively through all subdirs
data_dir = "../../../gnode/experiment_analysis/dat/"
meta_df = utl.all_unit_metadata(data_dir, reload=False)
meta_df = utl.load_spikes(meta_df)
meta_df = utl.default_filter(meta_df, trim=False)
meta_df = utl.merge_blocks(meta_df)
meta_df.tail()

2023-08-11 18:19:01,459 | DEBUG    | its_utility > all_unit_metadata > Found 59 hdf5 files in /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat
Fetching metadata from sessions: 100%|██████████| 59/59 [00:00<00:00, 70.84it/s]
2023-08-11 18:19:02,316 | DEBUG    | its_utility > load_spikes > Loading spikes for 12584 units, 31190 rows for pandas dataframe.
Loading spikes for sessions: 100%|██████████| 58/58 [00:36<00:00,  1.58it/s]
2023-08-11 18:19:39,100 | WARNING  | its_utility > default_filter > Some units already have invalid_spiketimes_check, we overwrite.
2023-08-11 18:19:39,109 | DEBUG    | its_utility > default_filter > Default quality checks, valid rows before: 31190
2023-08-11 18:19:39,115 | DEBUG    | its_utility > default_filter > After rate check: 31120
2023-08-11 18:19:39,124 | DEBUG    | its_utility > default_filter > After zero-length check: 31120
2023-08-11 18:19:39,189 | DEBUG    | its_utility > default_filter > After minmum-duration ch

,session,stimulus,block,unit_id,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes
43684,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190716,LP,SUCCESS,1067.306152,3.486347,/Users/paul/para/2_Projects/information_timesc...,3721,"[<xarray.DataArray ()>\narray(3.1584473, dtype..."
43685,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190722,LP,SUCCESS,1077.341553,2.770709,/Users/paul/para/2_Projects/information_timesc...,2985,"[<xarray.DataArray ()>\narray(0.40185547, dtyp..."
43686,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190724,LP,SUCCESS,1076.488525,2.043682,/Users/paul/para/2_Projects/information_timesc...,2200,"[<xarray.DataArray ()>\narray(1.0895996, dtype..."
43687,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190819,LP,SUCCESS,1075.182861,2.104758,/Users/paul/para/2_Projects/information_timesc...,2263,"[<xarray.DataArray ()>\narray(1.2353516, dtype..."
43688,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190848,VISrl,SUCCESS,1071.963867,0.806930,/Users/paul/para/2_Projects/information_timesc...,865,"[<xarray.DataArray ()>\narray(1.5703125, dtype..."


In [31]:
utl.log.setLevel("DEBUG")
meta_df

,session,stimulus,block,unit_id,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,spiketimes
0,787025148,natural_movie_one_more_repeats,3.0,951013153,VISam,SUCCESS,900.668806,21.542880,/Users/paul/para/2_Projects/information_timesc...,19403,[[[<xarray.DataArray (spiketimes: 30719)>\narr...
1,787025148,natural_movie_one_more_repeats,3.0,951013143,VISam,SUCCESS,900.718039,11.652925,/Users/paul/para/2_Projects/information_timesc...,10496,[[[<xarray.DataArray (spiketimes: 30719)>\narr...
2,787025148,natural_movie_one_more_repeats,3.0,951013133,VISam,SUCCESS,894.867666,0.448111,/Users/paul/para/2_Projects/information_timesc...,401,[[[<xarray.DataArray (spiketimes: 30719)>\narr...
3,787025148,natural_movie_one_more_repeats,3.0,951013202,VISam,SUCCESS,900.265072,2.384853,/Users/paul/para/2_Projects/information_timesc...,2147,[[[<xarray.DataArray (spiketimes: 30719)>\narr...
4,787025148,natural_movie_one_more_repeats,3.0,951013187,VISam,SUCCESS,896.074534,0.706414,/Users/paul/para/2_Projects/information_timesc...,633,[[[<xarray.DataArray (spiketimes: 30719)>\narr...
...,...,...,...,...,...,...,...,...,...,...,...
43684,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190716,LP,SUCCESS,1067.306152,3.486347,/Users/paul/para/2_Projects/information_timesc...,3721,"[<xarray.DataArray ()>\narray(3.1584473, dtype..."
43685,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190722,LP,SUCCESS,1077.341553,2.770709,/Users/paul/para/2_Projects/information_timesc...,2985,"[<xarray.DataArray ()>\narray(0.40185547, dtyp..."
43686,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190724,LP,SUCCESS,1076.488525,2.043682,/Users/paul/para/2_Projects/information_timesc...,2200,"[<xarray.DataArray ()>\narray(1.0895996, dtype..."
43687,847657808,natural_movie_one_more_repeats,merged_3.0_and_8.0,951190819,LP,SUCCESS,1075.182861,2.104758,/Users/paul/para/2_Projects/information_timesc...,2263,"[<xarray.DataArray ()>\narray(1.2353516, dtype..."


In [3]:
analysed_df = pd.read_hdf("/Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/meta_df_final.h5", "/meta_df")
analysed_df

,,,,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,R_tot,tau_R,tau_single,tau_double,tau_R_details,tau_single_details,tau_double_details
unit_id,stimulus,session,block,,,,,,,,,,,,,
950987325,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.087879,1.999804,/data.nst/share/data/allen_visual_coding_neuro...,1796,0.028897,0.292095,2.204573,2.204866,"{'firing_rate': 1.9997995746528745, 'firing_ra...","{'tau': 2.2045730546241966, 'mre': 0.997734557...","{'tau': 2.2048661904516664, 'mre': 0.997734858..."
950987344,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,898.868180,0.837720,/data.nst/share/data/allen_visual_coding_neuro...,753,0.254426,0.016040,0.049767,0.045802,"{'firing_rate': 0.8332684370376139, 'firing_ra...","{'tau': 0.04976678697174655, 'mre': 0.90441349...","{'tau': 0.04580227717841129, 'mre': 0.89658257..."
950987340,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.287381,1.568388,/data.nst/share/data/allen_visual_coding_neuro...,1412,0.039074,0.057306,0.263706,0.233516,"{'firing_rate': 1.5594974952515301, 'firing_ra...","{'tau': 0.26370610494040975, 'mre': 0.98121811...","{'tau': 0.23351615919178326, 'mre': 0.97881581..."
950987352,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,894.181973,0.542395,/data.nst/share/data/allen_visual_coding_neuro...,485,0.074347,0.126436,0.657249,0.594182,"{'firing_rate': 0.5412750157965074, 'firing_ra...","{'tau': 0.6572492090313748, 'mre': 0.992421399...","{'tau': 0.5941821956040216, 'mre': 0.991620379..."
950987362,natural_movie_one_more_repeats,774875821,3.0,VISam,SUCCESS,900.613482,2.053045,/data.nst/share/data/allen_visual_coding_neuro...,1849,0.037868,0.110199,0.778819,0.749355,"{'firing_rate': 2.036386247175541, 'firing_rat...","{'tau': 0.7788194682871189, 'mre': 0.993600590...","{'tau': 0.7493554822530509, 'mre': 0.993349810..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
951190716,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1067.306152,3.486347,/data.nst/share/data/allen_visual_coding_neuro...,3721,0.089890,0.062295,0.148944,0.135397,"{'firing_rate': 3.4038845321415523, 'firing_ra...","{'tau': 0.1489437713720214, 'mre': 0.966987495...","{'tau': 0.13539671566484524, 'mre': 0.96374502..."
951190722,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1077.341553,2.770709,/data.nst/share/data/allen_visual_coding_neuro...,2985,0.059516,0.130128,0.308041,0.250979,"{'firing_rate': 2.7539924536708296, 'firing_ra...","{'tau': 0.3080412633079295, 'mre': 0.983899430...","{'tau': 0.25097930824317566, 'mre': 0.98027516..."
951190724,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1076.488525,2.043682,/data.nst/share/data/allen_visual_coding_neuro...,2200,0.078478,0.062421,0.348126,0.315869,"{'firing_rate': 2.029744818809278, 'firing_rat...","{'tau': 0.34812607538185547, 'mre': 0.98574003...","{'tau': 0.31586908863716606, 'mre': 0.98429528..."


In [11]:
meta_df.set_index(analysed_df.index.names, inplace=True, drop=True)

In [17]:
meta_df = utl.strict_merge_dfs_by_index(meta_df, analysed_df)
meta_df.reset_index(inplace=True, drop=False)

2023-08-11 17:03:36,785 | WARNING  | its_utility > strict_merge_dfs_by_index > Column filepath differs in 23293 rows.


In [34]:
meta_df["stimulus"].unique()

array(['natural_movie_one_more_repeats', 'spontaneous'], dtype=object)

In [32]:
utl.log.setLevel("DEBUG")
utl.load_metrics(meta_df, data_dir)


2023-08-11 17:37:58,845 | DEBUG    | its_utility > load_metrics > Loaded columns ['unit_id', 'g_dsi_dg', 'mod_idx_dg', 'image_selectivity_ns'] from /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/brain_observatory_unit_metrics_filtered.csv
2023-08-11 17:37:59,046 | DEBUG    | its_utility > load_metrics > Loaded columns ['unit_id', 'g_dsi_dg', 'mod_idx_dg'] from /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/functional_connectivity_analysis_metrics.csv
2023-08-11 17:37:59,046 | INFO     | its_utility > load_metrics > Column g_dsi_dg found in multiple dataframes.
2023-08-11 17:37:59,047 | INFO     | its_utility > load_metrics > Column mod_idx_dg found in multiple dataframes.
2023-08-11 17:37:59,049 | DEBUG    | its_utility > load_metrics > Matched 0 rows from meta_df in /Users/paul/para/2_Projects/information_timescales/repo/gnode/experiment_analysis/dat/brain_observatory_unit_metrics_filtered.csv
2023-08-11 17:

,unit_id,stimulus,session,block,ecephys_structure_acronym,invalid_spiketimes_check,recording_length,firing_rate,filepath,num_spikes,...,R_tot,tau_R,tau_R_details,tau_double,tau_double_details,tau_single,tau_single_details,g_dsi_dg,image_selectivity_ns,mod_idx_dg
0,951013153,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.668806,21.542880,/Users/paul/para/2_Projects/information_timesc...,19403,...,0.033328,0.011799,"{'firing_rate': 21.52952801803102, 'firing_rat...",0.055968,"{'tau': 0.05596782558158293, 'mre': 0.91453728...",1.790838,"{'tau': 1.7908381118145253, 'mre': 0.997211905...",NaN,NaN,NaN
1,951013143,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.718039,11.652925,/Users/paul/para/2_Projects/information_timesc...,10496,...,0.075185,0.056762,"{'firing_rate': 10.611510791366905, 'firing_ra...",0.594005,"{'tau': 0.5940050646529661, 'mre': 0.991617890...",-161.079972,"{'tau': -161.0799718101262, 'mre': 1.000031040...",NaN,NaN,NaN
2,951013133,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,894.867666,0.448111,/Users/paul/para/2_Projects/information_timesc...,401,...,0.074266,0.244077,"{'firing_rate': 0.448109781309017, 'firing_rat...",0.496862,"{'tau': 0.49686180680147624, 'mre': 0.98998730...",1.120946,"{'tau': 1.1209462692699896, 'mre': 0.995549416...",NaN,NaN,NaN
3,951013202,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,900.265072,2.384853,/Users/paul/para/2_Projects/information_timesc...,2147,...,0.085065,0.162141,"{'firing_rate': 2.3848533487362054, 'firing_ra...",1.613602,"{'tau': 1.613601510647753, 'mre': 0.9969061374...",1.906939,"{'tau': 1.9069389092919093, 'mre': 0.997381431...",NaN,NaN,NaN
4,951013187,natural_movie_one_more_repeats,787025148,3.0,VISam,SUCCESS,896.074534,0.706414,/Users/paul/para/2_Projects/information_timesc...,633,...,0.229616,0.012902,"{'firing_rate': 0.7030661495968529, 'firing_ra...",0.029344,"{'tau': 0.029344318113069354, 'mre': 0.8433352...",0.040537,"{'tau': 0.04053663662381895, 'mre': 0.88395845...",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23288,951190716,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1067.306152,3.486347,/Users/paul/para/2_Projects/information_timesc...,3721,...,0.089890,0.062295,"{'firing_rate': 3.4038845321415523, 'firing_ra...",0.135397,"{'tau': 0.13539671566484524, 'mre': 0.96374502...",0.148944,"{'tau': 0.1489437713720214, 'mre': 0.966987495...",NaN,NaN,NaN
23289,951190722,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1077.341553,2.770709,/Users/paul/para/2_Projects/information_timesc...,2985,...,0.059516,0.130128,"{'firing_rate': 2.7539924536708296, 'firing_ra...",0.250979,"{'tau': 0.25097930824317566, 'mre': 0.98027516...",0.308041,"{'tau': 0.3080412633079295, 'mre': 0.983899430...",NaN,NaN,NaN
23290,951190724,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1076.488525,2.043682,/Users/paul/para/2_Projects/information_timesc...,2200,...,0.078478,0.062421,"{'firing_rate': 2.029744818809278, 'firing_rat...",0.315869,"{'tau': 0.31586908863716606, 'mre': 0.98429528...",0.348126,"{'tau': 0.34812607538185547, 'mre': 0.98574003...",NaN,NaN,NaN
23291,951190819,natural_movie_one_more_repeats,847657808,merged_3.0_and_8.0,LP,SUCCESS,1075.182861,2.104758,/Users/paul/para/2_Projects/information_timesc...,2263,...,0.018823,0.244690,"{'firing_rate': 2.1010337755828066, 'firing_ra...",0.404400,"{'tau': 0.4043998633728332, 'mre': 0.987712119...",0.415153,"{'tau': 0.4151534238035432, 'mre': 0.988028495...",NaN,NaN,NaN
